In [ ]:
import array
import random
import time
import json
import os
from joblib import dump, load
import numpy as np
from keras.models import load_model
from math import sqrt
from deap import algorithms
from deap import base
from deap import benchmarks
from deap import creator
from deap import tools
from deap.benchmarks.tools import diversity, convergence, hypervolume

import plotly.graph_objects as go
import pandas as pd


#define the folder for the code
path=r'xxx'
os.chdir(path)


class Inputs():
    #define the input parameters
    def __init__(self, coil_position,crucible_position,crucible_wall,crucible_bottom,crucible_height,solvent_radius,solvent_height):
        self.coil_position = coil_position
        self.crucible_position = crucible_position
        self.crucible_wall = crucible_wall
        self.crucible_bottom = crucible_bottom
        self.crucible_height = crucible_height
        self.solution_height=solvent_height
        self.solution_length=solvent_radius 
        
    #generate the input data for machine learning model
    def make_condition(self):
        cond = [self.coil_position, self.crucible_position, self.crucible_wall, self.crucible_bottom, 
                self.crucible_height,self.solution_length,self.solution_height]
        #In this code, x and y is the coordinate position inside solvent
        x_num, y_num = self.x_y_num()
        x,y=self.x_y(x_num, y_num)
        X_Y_melt = self.XY_matrix(x, y, x_num, y_num)
        #add the coordinate position as input data for machine learning model        
        condition = []
        for i in range(x_num*y_num):
            c_ = np.copy(cond)
            condition.append(c_)
        condition = np.vstack(condition)
        condition = np.hstack((condition, X_Y_melt))
        return condition
    #Generate number of the coordinate position
    def x_y_num(self):
        x_num = int(self.solution_length)*2+1
        y_num = int(self.solution_height)*2+1
        x_num = int(x_num)
        y_num = int(y_num)
        return x_num, y_num
    #Linespace the coordinate position 
    def x_y(self, x_num, y_num):   
        x = np.linspace(0, int(self.solution_length), x_num).T
        y = np.linspace(0, int(self.solution_height), y_num).T
        return x, y
    #Generate matrix of the coordinate position
    def XY_matrix(self, x, y, x_num, y_num):
        XY = []       
        for i in range(y_num):
            XY_ = []
            for j in range(x_num):
                x_ = x[j]
                y_ = self.crucible_bottom_thickness+y[i]
                xy_ = np.hstack((x_, y_))
                XY_.append(xy_)
            XY_ = np.vstack(XY_)
            XY.append(XY_)
        XY = np.vstack(XY)
        return XY
    #load preprocess model
    def stsc(self): 
        x_st = load("stsc/stsc_x.joblib")
        y_st = load("stsc/stsc_y.joblib")
        return x_st, y_st


class NN(Inputs):
    def __init__(self):
        super(Inputs, self).__init__()
        
    #load machine learning model
    def Load_model(self):
        model = load_model(self.model_path)
        return model

    #predict output data
    def Predict(self, model, individual):
        x_st, y_st = self.stsc()
        individual = np.array(individual)
        individual = np.append(individual, [1,1,1])

        # generate input data
        input = list(x_st.inverse_transform(individual))
        inputs = Inputs(coil_position=int(individual[0]), crucible_position=int(individual[1]),crucible_wall=int(individual[2]),
                        crucible_bottom=int(individual[3]), crucible_height=int(individual[4]),solvent_radius=int(individual[5]),
                        solvent_height=int(individual[6]))
        INPUT = inputs.make_condition()
        OriginInput=INPUT

        # generate output data
        INPUT = x_st.transform(INPUT)
        pred = model.predict(INPUT)
        
        individual = list(individual)
        individual.pop()
        individual.pop()
        individual.pop()
        
        return pred,OriginInput

#define objective function
class Objective_function(NN):
    def __init__(self, individual):
        self.individual = individual
        super().__init__()
    
    # main_function
    def evaluate(self, model, individual):
        nn = NN()
        pred,OriginInput = nn.Predict(model=model, individual=individual)
        y_st = load("stsc/stsc_y.joblib")
        pred = y_st.inverse_transform(pred)
        
        #input & output data
        resultpred=np.hstack([OriginInput,pred])
        
        #define temperature and carbon concentration
        result_T=resultpred[:,9]
        result_num=result_x.shape[0]
        result_carbon=resultpred[:,12]
                
        growth_rate=[]
        erosion_rate=[]
        polycrystal_rate=[]
        
        for i in range(result_num):
            
            #find solvent surface
            if result_y[i-1]==resultpred[0][3]+resultpred[0][4]:
                # find the crystal surface
                if result_x[i-1]<75:
                    #equibrim carbon concentration
                    Ceq=np.exp(-29819.53701567/result_T[i-1]+6.70581678)
                    #C_rate=A_0 (C-C_e )exp((-E_0)/RT) 
                    C_rate_growth = 32*(result_carbon[i-1] - Ceq)*np.exp(-125000/8.314/result_T[i-1])
                    #v_SiC=m_sic/(m_c ρ_SiC ) C_rate
                    growth_rate_single=40/12/3216*C_rate_growth
                    #include all growth rate
                    growth_rate.append(growth_rate_single)
                    
            #find crucible sidewall
            if result_x[i-1]==int(resultpred[0][5]):
                #equibrim carbon concentration
                Ceq=np.exp(-29819.53701567/result_T[i-1]+6.70581678)
                #C_rate=A_0 (C-C_e )exp((-E_0)/RT) 
                C_rate_erosion = 32*(result_carbon[i-1] - Ceq)*np.exp(-125000/8.314/result_T[i-1])
                # v_erosion=C_rate⁄ρ_C
                erosion_rate_single=1/1950*C_rate_erosion
                #include all erosion rate
                erosion_rate.append(erosion_rate_single) 
            
            #find crucible bottom
            if 0.1%float(result_y[i-1])==0.1%float(resultpred[0][3]):
                #equibrim carbon concentration
                Ceq=np.exp(-29819.53701567/result_T[i-1]+6.70581678)
                #C_rate=A_0 (C-C_e )exp((-E_0)/RT) 
                C_rate_polycrystal = 32*(result_carbon[i-1] - Ceq)*np.exp(-125000/8.314/result_T[i-1])
                #v_poly=α m_sic/(m_c ρ_SiC ) C_rate	
                polycrystal_rate_single=1.9*40/12/3216*C_rate_polycrystal
                #include all growth rate
                polycrystal_rate.append(polycrystal_rate_single)
                                        
        
        #define objective functions- standard deviation
        F1 = np.std(growth_rate)
        F1 = np.std(erosion_rate)
        F1 = np.std(polycrystal_rate)

        return F1,F2,F3
     

def evaluate(individual, model):
    Ob_func = Objective_function(individual=individual)
    F1,F2,F3 = Ob_func.evaluate(model=model, individual=individual)
    return F1,F2,F3,

def uniform(low, up, size=None):
    try:
        return [random.uniform(a, b) for a, b in zip(low, up)]
    except TypeError:
        return [random.uniform(a, b) for a, b in zip([low] * size, [up] * size)]


creator.create("FitnessMin", base.Fitness, weights=(-1.0,-1.0,-1.0))
creator.create("Individual", list, typecode='d', fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# define the range for input data in genetic algrithom
BOUND_LOW, BOUND_UP = [0]+[0]+[10]+[20]+[160]+[90]+[20],[100]+[80]+[50]+[40]+[230]+[230]+[60]



NDIM = 7
model_path = "result/model.h5"
model = load_model(model_path)

toolbox.register("attr_float", uniform, BOUND_LOW, BOUND_UP, NDIM)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_float)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate, model=model)
toolbox.register("mate", tools.cxSimulatedBinaryBounded, low=BOUND_LOW, up=BOUND_UP, eta=20.0)
toolbox.register("mutate", tools.mutPolynomialBounded, low=BOUND_LOW, up=BOUND_UP, eta=20.0, indpb=1.0 / NDIM)
toolbox.register("select", tools.selNSGA2)

def main():
    print("=======  start - NSGA2  ========")

    # generation
    NGEN = 1000
    # population
    MU = 5000
    # crossover
    CXPB = 0.9

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean, axis=0)
    stats.register("std", np.std, axis=0)
    stats.register("min", np.min, axis=0)
    stats.register("max", np.max, axis=0)

    logbook = tools.Logbook()
    logbook.header = "gen", "evals", "std", "min", "avg", "max"

    pop = toolbox.population(n=MU)

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in pop if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # This is just to assign the crowding distance to the individuals
    # no actual selection is done
    pop = toolbox.select(pop, len(pop))

    record = stats.compile(pop)
    logbook.record(gen=0, evals=len(invalid_ind), **record)
    print(logbook.stream)

    # Begin the generational process
    for gen in range(1, NGEN):
        # Vary the population
        print("------- Gneration %f -------" % gen)
        start = time.time()
        offspring = tools.selTournamentDCD(pop, len(pop))
        offspring = [toolbox.clone(ind) for ind in offspring]

        for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
            if random.random() <= CXPB:
                toolbox.mate(ind1, ind2)

            toolbox.mutate(ind1)
            toolbox.mutate(ind2)
            del ind1.fitness.values, ind2.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Select the next generation population
        pop = toolbox.select(pop + offspring, MU)
        record = stats.compile(pop)
        logbook.record(gen=gen, evals=len(invalid_ind), **record)
        end = time.time() - start
        print(logbook.stream)
        print("Time duration[s]:", end)

    return pop, logbook


if __name__ == "__main__":
    pop, stats = main()
    pop.sort(key=lambda x: x.fitness.values)
    
    front = np.array([ind.fitness.values for ind in pop])
    result = np.hstack([pop,front])
    np.savetxt("pareto_front.csv", result, delimiter=',')

